In [0]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from pyspark.sql.functions import col

# --- 1. DATA PREPARATION ---
df_kpi = spark.read.table("hive_metastore.default.gold_business_kpis").toPandas()
df_preds = spark.read.table("hive_metastore.default.gold_patient_predictions") \
    .orderBy(col("admission_ts").desc()).limit(300).toPandas()

# Clean labels (No Emojis)
df_preds["risk_clean"] = df_preds["risk_alert"].str.replace("🔴 ", "").str.replace("🟠 ", "").str.replace("🟢 ", "")

# Prepare Heatmap Data
df_melted = df_kpi.melt(
    id_vars=["department"], 
    value_vars=["2_morning_shift_admits", "3_evening_shift_admits", "4_night_shift_admits"],
    var_name="Shift", value_name="Patients"
)
df_melted["Shift"] = df_melted["Shift"].str.replace("_shift_admits", "").str.replace("[0-9]_", "", regex=True).str.title()
heatmap_data = df_melted.pivot(index='department', columns='Shift', values='Patients')

# --- 2. BUILD LAYOUT (Standard 2x2 Grid) ---
# We use 'xy' for everything to prevent errors
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Critical Bed Blocks by Dept (>48h)", "AI Risk Radar: Predicted Stay vs Age", 
                    "Staffing Heatmap (Shift Load)", "Department Efficiency (Bubble Size = Volume)"),
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# --- CHART 1 (Top-Left): Critical Blocks (Bar Chart) ---
# Replaces the Gauge. Shows exactly WHICH dept is blocked.
fig.add_trace(go.Bar(
    x=df_kpi["department"],
    y=df_kpi["12_critical_stay_over_2_days"],
    name="Blocked Beds",
    marker_color="darkred"
), row=1, col=1)

# --- CHART 2 (Top-Right): AI Prediction Scatter ---
color_map = {"CRITICAL: Likely Long Term Stay": "#d62728", "HIGH: Monitor Closely": "#ff7f0e", "LOW: Routine Discharge": "#2ca02c"}
colors = df_preds["risk_clean"].map(color_map)

fig.add_trace(go.Scatter(
    x=df_preds["age"],
    y=df_preds["predicted_los_hours"],
    mode='markers',
    marker=dict(size=8, color=colors, opacity=0.7),
    text=df_preds["department"],
    name="Patient Risk"
), row=1, col=2)

# Add Critical Threshold Line (96h)
fig.add_hline(y=96, line_dash="dot", line_color="red", annotation_text="Critical (96h)", row=1, col=2)

# --- CHART 3 (Bottom-Left): Staffing Heatmap ---
fig.add_trace(go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='Viridis',
    texttemplate="%{z}",
    name="Staff Load"
), row=2, col=1)

# --- CHART 4 (Bottom-Right): Efficiency Bubble Chart ---
fig.add_trace(go.Scatter(
    x=df_kpi["14_complexity_score"],
    y=df_kpi["6_avg_los_hours"],
    mode='markers',
    text=df_kpi["department"],
    marker=dict(
        size=df_kpi["1_total_admissions"],  # Bubble size = Volume
        sizemode='area',
        sizeref=2.*max(df_kpi["1_total_admissions"])/(40.**2), # Scaling factor
        color=df_kpi["13_bed_turnaround_rate"],
        colorscale="RdYlGn_r",
        showscale=True,
        colorbar=dict(title="Turnaround", x=1.0, len=0.4, y=0.2)
    ),
    name="Efficiency"
), row=2, col=2)

# --- 3. FINAL POLISH ---
fig.update_layout(
    title_text="<b>Hospital Operations Command Center</b>",
    title_x=0.5,
    height=800,
    width=1200,
    template="plotly_white",
    showlegend=False
)

fig.show()